# Исследование пользователей двух городов

## Описание

Сравнение Москвы и Петербурга окружено мифами. Например:
 * Москва — мегаполис, подчинённый жёсткому ритму рабочей недели;
 * Петербург — культурная столица, со своими вкусами.

На данных Яндекс.Музыки мы сравним поведение пользователей двух столиц.

**Цель исследования** — проверка трех гипотез:
1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города. 
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

## Обзор данных

Изучим данные сервиса Яндекс.Музыка.

In [1]:
# импортируем библиотеку pandas
import pandas as pd

Прочитаем файл yandex_music_project.csv и сохраним его в переменной df.

In [2]:
df = pd.read_csv('yandex_music_project.csv')

Выведем на экран первые десять строк таблицы.

In [3]:
df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


Посмотрим общую информацию о таблице c помощью метода info().

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


Итак, в таблице семь столбцов. Тип данных во всех столбцах — `object`.

Согласно документации к данным:
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

Количество значений в столбцах различается. Следовательно, в данных есть пропущенные значения.

**Вывод**

В каждой строке таблицы содержатся данные о прослушанном треке (название, исполнитель, жанр), а также данные о пользователе (из какого он города, в какое время и день недели он слушал музыку). 

Согласно общей информации, в данных есть пропуски и некачественные названия столбцов. Для проверки гипотез нам особенно ценны столбцы time, Day и City, а данные из столбца genre позволят узнать самые популярные жанры.

## Предобработка данных

Исправим стиль в заголовках столбцов и исключим пропуски. Затем проверим данные на дубликаты.

In [5]:
# выведем на экран названия столбцов таблицы df
print(df.columns)

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')


В названиях столбцов есть пробелы, которые могут затруднять доступ к данным.

Переименуем столбцы для удобства дальнейшей работы и проверим результат.

In [6]:
# переименование столбцов
df = df.rename(columns={'  userID':'user_id', 'Track':'track', '  City  ':'city', 'Day':'day'})

In [7]:
# проверка результатов - перечень названий столбцов
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

Сначала посчитаем: сколько в таблице пропущенных значений. Для этого достаточно двух методов pandas.

In [8]:
# подсчёт пропусков
df.isna().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Пустые значения свидетельствуют, что для некоторых треков доступна не вся информация. Причины могут быть разные: вероятно не назван конкретный исполнитель народной песни; или же если проблемы с записью данных. Каждый отдельный случай необходимо разобрать и выявить причину. 

В столбцах track и artist пропуски не важны для дальнейшего анализа данных, поэтому их будет достаточно заменить явными обозначениями. А вот пропуски в столбце genre могут помешать сравнению музыкальных вкусов в Москве и Санкт-Петербурге. Но на данный момент у нас нет возможности установить причину пропусков и восстановить данные.

Заменим пропущенные значения в столбцах track, artist и genre на строку 'unknown'. Для этого создадим список columns_to_replace, переберем его элементы циклом for и для каждого столбца выполним замену пропущенных значений.

In [9]:
# перебор названий столбцов в цикле и замена пропущенных значений на 'unknown'
columns_to_replace = ['track', 'artist', 'genre']
for column in columns_to_replace:
    df[column] = df[column].fillna('unknown')

Убедимся, что в таблице не осталось пропусков. Для этого ещё раз посчитаем пропущенные значения.

In [10]:
# подсчёт пропусков
df.isna().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

Посчитайте явные дубликаты в таблице.

In [11]:
# подсчёт явных дубликатов
df.duplicated().sum()

3826

Используем специальный метод pandas, чтобы удалить явные дубликаты.

In [12]:
# удаление явных дубликатов
df = df.drop_duplicates()

Ещё раз посчитаем явные дубликаты в таблице, чтобы убедится в их отсутствии.

In [13]:
# проверка на отсутствие дубликатов
df.duplicated().sum()

0

Теперь проверим столбец genre на наличие неявных дубликатов. Для начала выведем на экран список уникальных названий жанров, отсортированный в алфавитном порядке.

In [14]:
# просмотр уникальных названий жанров
df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

При просмотре списока музыкальных жанров мы нашли неявные дубликаты названия hiphop:
- hip,
- hop,
- hip-hop.

Очистим от них таблицу, используя метод replace() с двумя аргументами: списком строк-дубликатов (включащий *hip*, *hop* и *hip-hop*) и строкой с правильным значением. Заменим каждое значение из списка дубликатов на hiphop.

In [15]:
# устранение неявных дубликатов
duplicates = ['hip', 'hop', 'hip-hop']
genre = 'hiphop'
df['genre'] = df['genre'].replace(duplicates, genre)

Проверим, что заменили неправильные названия на hiphop. Для этого выведем отсортированный список уникальных значений столбца genre.

In [16]:
# проверка на неявные дубликаты
df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

**Вывод**

В предобработке данных мы изменили названия столбцов в таблице, чтобы проще было в дальнейшем работать. Удалили явные дубликаты, а также провели замену неявных дубликатов жанра hiphop в столбце genre. Пропущенные значения мы заменили на 'unknown'. 

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверим это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. 

Для этого нам надо разделить пользователей Москвы и Санкт-Петербурга, а также сравнить: сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.

In [17]:
# подсчёт прослушиваний в каждом городе
df.groupby('city')['user_id'].count()

city
Moscow              42741
Saint-Petersburg    18512
Name: user_id, dtype: int64

Количество прослушиваний в Москве составляет 42 741, а в Петербурге - 18 512. Вероятно, это связано с тем, что самих пользователей в Москве больше, чем в Санкт-Петербурге.

Теперь сгруппируем данные по дню недели и посчитаем прослушивания в понедельник, среду и пятницу. 

In [18]:
# подсчёт прослушиваний в каждый из трёх дней
df.groupby('day')['user_id'].count()

day
Friday       21840
Monday       21354
Wednesday    18059
Name: user_id, dtype: int64

Меньше всего пользователи из двух городов слушают музыку по средам. Но необходимо рассмотреть каждый город в отдельности.

Создадим функцию number_tracks(), которая принимает как параметры день недели и название города, а возвращает количество прослушанных композиций, для которых известен жанр. 

In [19]:
# создание функции number_tracks()
def number_tracks(day, city):
    track_list = df[(df['day'] == day)&(df['city'] == city)]
    track_list_count = track_list['user_id'].count()
    return track_list_count 
# в переменной track_list сохраняются те строки таблицы df, для которых 
# значение в столбце 'day' равно параметру day и одновременно значение в столбце 'city' равно параметру city 
# в переменной track_list_count сохраняется число значений столбца 'user_id', 
# рассчитанное методом count() для таблицы track_list
# функция возвращает число - значение track_list_count

С помощью функции number_tracks() проверим количество прослушанных композиций для каждого города и понедельника, затем среды и пятницы.

In [20]:
# количество прослушиваний в Москве по понедельникам
number_tracks('Monday', 'Moscow')

15740

In [21]:
# количество прослушиваний в Санкт-Петербурге по понедельникам
number_tracks('Monday', 'Saint-Petersburg')

5614

In [22]:
# количество прослушиваний в Москве по средам
number_tracks('Wednesday', 'Moscow')

11056

In [23]:
# количество прослушиваний в Санкт-Петербурге по средам
number_tracks('Wednesday', 'Saint-Petersburg')

7003

In [24]:
# количество прослушиваний в Москве по пятницам
number_tracks('Friday', 'Moscow')

15945

In [25]:
# количество прослушиваний в Санкт-Петербурге по пятницам
number_tracks('Friday', 'Saint-Petersburg')

5895

Создадим c помощью конструктора pd.DataFrame таблицу, где названия колонок — это ['city', 'monday', 'wednesday', 'friday'], а данные — это результаты, которые были получены с помощью number_tracks.

In [26]:
# таблица с результатами
info = pd.DataFrame(data=[['Москва', 15740, 11056, 15945], ['Санкт-Петербург', 5614, 7003, 5895]], 
                    columns=['city', 'monday', 'wednesday', 'friday'])# Таблица с результатами
info

,city,monday,wednesday,friday
0,Москва,15740,11056,15945
1,Санкт-Петербург,5614,7003,5895


**Вывод**

Результаты показывают, что в Москве активно слушают музыку по понедельникам и пятницам, а в среду заметено меньше. В Петербурге же, наоборот, больше слушают музыку по средам, нежели в понедельник и пятницу. Следовательно, можно подтвердить первую гипотезу, в которой говорится, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге.

### Музыка в начале и в конце недели

Теперь проверим вторую гипотезу, которая гласит, что утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие; так же и вечером пятницы преобладают разные жанры — в зависимости от города.

Сохраним данные по Москве в переменную moscow_general, а данные по Санкт-Петербургу — в spb_general.

In [27]:
# получение таблицы moscow_general из тех строк таблицы df, для которых значение в столбце 'city' равно 'Moscow'
moscow_general = df[df['city'] == 'Moscow']

In [28]:
# получение таблицы spb_general из тех строк таблицы df, для которых значение в столбце 'city' равно 'Saint-Petersburg'
spb_general = df[df['city'] == 'Saint-Petersburg']

Создадим функцию genre_weekday(), которая возвращает список жанров по запрошенному дню недели и времени суток с такого-то часа по такой-то.

In [29]:
# объявление функции genre_weekday() с параметрами table, day, time1, time2
def genre_weekday(df, day, time1, time2):
    genre_df = df[df['day'] == day]
    genre_df = genre_df[genre_df['time'] < time2]
    genre_df = genre_df[genre_df['time'] > time1]
    genre_df_grouped = genre_df.groupby('genre')['genre'].count()
    genre_df_sorted = genre_df_grouped.sort_values(ascending=False)
    return genre_df_sorted[:10]
# в переменной genre_list сохраняются те строки df, для которых одновременно:
# 1) значение в столбце 'weekday' равно параметру day,
# 2) значение в столбце 'time' больше time1 и
# 3) меньше time2.
# в переменной genre_list_sorted сохраняются в порядке убывания  
# первые 10 значений Series, полученной подсчётом числа значений 'genre_name'
# сгруппированной по столбцу 'genre_name' таблицы genre_list
# функция возвращает значение genre_list_sorted

Теперь сравним результаты функции genre_weekday() для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00).

In [30]:
# вызов функции для утра понедельника в Москве (вместо df — таблица moscow_general)
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

In [31]:
# вызов функции для утра понедельника в Петербурге (вместо df — таблица spb_general)
genre_weekday(spb_general, 'Monday', '07:00', '11:00') 

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [32]:
# вызов функции для вечера пятницы в Москве
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

In [33]:
# вызов функции для вечера пятницы в Петербурге
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

**Вывод**

При сравнении топ-10 жанров в понедельник утром и вечером в пятницу, можно сделать такие выводы:

1. Жанр pop является безусловным лидером в обоих городах.
2. Утром в понедельник в обоих городах слушают похожую музыку. Единственное отличие заключается в том, что в московский рейтинг музыкальных жанров вошёл “world”, а в петербургский — джаз и классика.
3. В Москве пропущенных значений в столбце genre оказалось так много, что значение 'unknown' заняло десятое место среди самых популярных жанров. Следовательно, пропущенные значения в этом столбце занимают существенную долю в данных и могут повлиять на достоверность исследования.
4. Вечер пятницы похож на утро понедельника в обоих городах. Только некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Поэтому, вторая гипотеза подтвердилась лишь частично: пользователи слушают похожую музыку в начале недели и в конце, а разница между Москвой и Петербургом не слишком выражена. Видно, что в конце каждого рейтинга для каждого города выделяются более характерные жанры, которые действительно меняют свои позиции в зависимости от дня недели и времени.

### Жанровые предпочтения в Москве и Петербурге

Далее проверим тертью гипотезу: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве;  а Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

Сгруппируем таблицу moscow_general по жанру и посчитаем прослушивания треков каждого жанра методом count(). Затем отсортируем результат в порядке убывания и сохраним его в таблице moscow_genres.

In [34]:
# группировка таблицы moscow_general по столбцу 'genre_name', 
# подсчёт числа значений 'genre_name' в этой группировке методом count(), 
# сортировка Series в порядке убывания и сохранение в moscow_genres
moscow_genres = moscow_general.groupby('genre')['genre'].count().sort_values(ascending=False) 

Выведем на экран первые десять строк moscow_genres.

In [35]:
# просмотр первых 10 строк moscow_genres
moscow_genres.head(10)

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

Теперь сгруппируем таблицу spb_general по жанру и посчитаем прослушивания треков каждого жанра. Результат отсортируем в порядке убывания и сохраним в таблице spb_genres.

In [36]:
# группировка таблицы spb_general по столбцу 'genre', 
# подсчёт числа значений 'genre' в этой группировке методом count(), 
# сортировка получившегося Series в порядке убывания и сохранение в spb_genres
spb_genres = spb_general.groupby('genre')['genre'].count().sort_values(ascending=False)

Выведем на экран первые десять строк spb_genres.

In [37]:
# просмотр первых 10 строк spb_genres
spb_genres.head(10)

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

**Вывод**

Гипотеза частично подтвердилась: в Москве, кроме абсолютно популярного жанра поп, есть направление русской популярной музыки. А рэп, вопреки предположению, одинаково популярен в Москве и Петербурге.

## Итоговый вывод

Мы проверили три гипотезы и установили:

1. Первая гипотеза (активность пользователей зависит от дня недели; причём в Москве и Петербурге это проявляется по-разному.) полностью подтвердилась. В Москве активно слушают музыку по понедельникам и пятницам, а в среду заметено меньше. В Петербурге же, наоборот, больше слушают музыку по средам, нежели в понедельник и пятницу.  


2. Вторая гипотеза, которая гласит: "В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города", подтвердилась лишь отчасти. Пользователи слушают похожую музыку в начале недели и в конце, а разница между Москвой и Петербургом не слишком выражена. Видно, что в конце каждого рейтинга для каждого города выделяются более характерные жанры, которые действительно меняют свои позиции в зависимости от дня недели и времени. Этот результат мог оказаться иным, если бы не пропуски в данных.


3. Третья гипотеза (Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп) частично подтвердилась. В Москве, кроме абсолютно популярного жанра поп, есть направление русской популярной музыки. А рэп, вопреки предположению, одинаково популярен в Москве и Петербурге.